## New 2/26/2024

## This serves as the core logic to extracting content from reliefweb

### Once this is finished, it replaces reliefweb_situation_reports

## Process Steps
1) get the last 500 disaster summaries from reliefweb
2) process the json and build a dataframe structured such that:
   1) each paragraph has its own row
   2) each identified reference url in that paragraph is parsed out along with other metadata
3) write resulting dataframe to postgres db - note this completely replaces the previously-existing table

## Cautions
Occasionally the reliefweb api call will return an error - to the effect that there's some bad chunk or something.
I've left that unhandled. Conflicting evidence as to whether it's an intermittent issue, maybe cause by rate limiting,
 or an issue related to a specific summary's format. Both seem to have been the case at various times.
 

In [1]:
import requests
import pandas as pd

import json
import os
import uuid
import re
import importlib

from datetime import datetime

import sys
sys.path.append('utilities')
import basic_utilities as utils

In [2]:
# key config vars
api_endpoint = 'https://api.reliefweb.int/v1/disasters?appname=amcross'


output_format = 'excel' #other option is 'postgres'
output_name = 'rw_disaster_summaries'

## If you plan to connect to a database
db_conf ={
    'host':"xxx",
    'port':'5432',
    'database':"postgres",
    'user':"postgres",
    'password':"xxx"
}



In [3]:
def get_rw_disaster_summaries(limit=500):
    


    #set a high limit for latest in case the job doesn't run for a long time
    params = {
    'appname': 'amcross','profile': 'full','preset': 'latest','limit': limit
    #,'filter[field]': "status",'filter[value]':'ongoing'
    ,'offset':0
    }

    # Make the API request
    response = requests.get(api_endpoint, params=params)
    
    # Check the status of the response
    if response.status_code == 200:
        # Parse and use the response data (in JSON format)
        data = response.json()
        return data['data']
        # for disasters if we don't return everything, can't get to the original api call
        #return data
    
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None



   



#get_rw_disaster_summaries(500)
         

In [4]:
def parse_summary_json(j):
    reference_url = ''
    file_url = ''
    themes = []
    author_org = ''
    
    rec_id = j['id']
    j = j['fields']

    status = j['status']
    glide_id = j.get('glide')
    status = j['status']
    title = j['name']
    description = j['description']
    file_url = j['url_alias']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    report_date = j['date']['changed']

    # 2 newlines will not necessarily reliably break out every paragraph
    # but it's better than the alternative where some sentences will get cut in half
    original_text_list = description.split("\n\n")
    idx_para=0
    for o in original_text_list:
        o = o.strip()
        if len(o) > 3:
            row = ['disaster summary',status,file_url,glide_id,rec_id,idx_para,primary_country,title,themes,o,reference_url,o,author_org,report_date]
            df_rw_disaster_sum.loc[len(df_rw_disaster_sum)] = row
            idx_para += 1


def parse_references(text):
    ref = ''
    org = ''
    date = ''
    iso_date = None
    url = ''
    
    pattern = r'\([^)]+\)\)\s?'  # Matches anything inside a ([xxxx](url)) pattern

    ref = re.findall(pattern, text)
    if len(ref) > 0:
        last_reference = ref[-1]
        #pattern = r'\[(.*?)[,|.]\s(\d+\s\w+\s\d+)\]\((.*?)\)'
        pattern = r'\[(.*?)[,|.]\s?(\d+\s\w+\s\d+)\]\((.*?)\)'

        # Use regex to find matches
        matches = re.search(pattern, last_reference)
        
        if matches:
            # Extracting the parts
            org = matches.group(1)
            date = matches.group(2)
            url = matches.group(3)
        

            try:
                date_object = datetime.strptime(date, "%d %b %Y") # 01 Jul 2023
                iso_date = date_object.date().isoformat()

            except:
                try:
                    date_object = datetime.strptime(date, "%d %B %Y") # 01 July 2023
                    iso_date = date_object.date().isoformat()
                except:
                    pass

            #date_object = datetime.strptime(date, "%d %b %Y")
           
            
        else:
            #TODO - some patterns don't get found... need to improve the pattern match
            pass
            #print(f"No match found. {ref}")

    
    return pd.Series({'references':ref,'auth_org':org,'date_str':date,'date_iso':iso_date,'reference_url':url})


In [5]:
# prepare receiving df
df_rw_disaster_sum = pd.DataFrame(columns = ['record_type','status','source_url','glide_id','doc_id','idx_para','source_level_country','source_title','source_desc',
                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])

disaster_data = get_rw_disaster_summaries(limit=200)

for summary in disaster_data:
    parse_summary_json(summary)

# now parse the idenfitied urls 
df_rw_disaster_sum['authoring_org'] = 'reliefweb'
df_rw_disaster_sum[['references','reference_auth_org','reference_date_str','reference_date_iso','reference_url']] = df_rw_disaster_sum['text'].apply(parse_references) 
df_rw_disaster_sum['para_id'] = 'rwdisastersumm_' + df_rw_disaster_sum['glide_id'].apply(lambda x: x.lower()) + '_' + df_rw_disaster_sum['idx_para'].astype(str)

## Data Persistence

In [6]:

if output_format == 'excel':
    outfile = utils.write_to_excel(df_rw_disaster_sum, filename=output_name)
    print(f"df written to: {outfile}")
elif output_format == 'postgres':
    persist_to_postgres(db_conf, output_name, df_rw_disaster_sum)
else:
    print(f"unknown persistence type: {output_format}")
          




df written to: stored_data\rw_disaster_summaries_64ae8b73482143a6b1361be7d3a124df.xlsx


In [57]:
df_rw_disaster_sum.shape

(1053, 18)